## MATH 157
# Post-Quantum Cryptography Pt. II
## By Delia McGrath

Sources: 
- https://defeo.lu/docet/assets/slides/2019-11-28-ntnu.pdf
- https://csrc.nist.gov/CSRC/media/Events/Second-PQC-Standardization-Conference/documents/accepted-papers/soukharev-pqdh-paper.pdf
- https://math.mit.edu/~drew/MITUMA2018.pdf
- https://eprint.iacr.org/2019/1321.pdf
- https://link.springer.com/content/pdf/10.1007/s11425-010-0053-3.pdf

## Isogeny cryptography

Before going into the details of isogeny, let's look at it in comparison to other post-quantum cryptographies.

<img src="KeySize.png" alt="keysize" width="300"/>

<img src="EncryptionPerformance.png" alt="encrypt" width="300"/>

The bar-graphs above show a surprising amount of encryption performance for isogenies, despite the size of their public keys. This is something to keep in mind as people look where to spend their time studying to be most successful in post-modern cryptography.

### Isogeny

What is it exactly?

$$\phi : P \rarr \phi(P)$$

A non-constant function, defined on an elliptic curve, that takes values on another elliptic curve and preserves point addition. (thus preserving elliptic structure)

Important to remember that elliptic curves have Group Law.
Group law: Three points on the line sum to zero.

#### SIDH

form of post-quantum Diffie-Hellman key agreement using isogenies


Every elliptic curve has a unique j invariant. We can think of this like a subset with multiple groupings.


SIDH works with prime fields where p = 3mod4 (for this example) 

This is often represented as $$F_{p^2} = F_p(i)$$ with $$i^2 + 1 = 0$$

Thus we are interested in the subset of p/12 +z, where z will depend on p mod 12
We'll have an element p that will have a subset of j-invariants


In [26]:
#we'll use a small p for our example
p = 431

431

In [25]:
ss = floor(p/12 + 2) #we'll floor this because we want a whole number for our invariant

37.0

<img src="SupersingularVariants.png" alt="pqdh" width="500"/>

()
Simply put, Alice and Bob will start out on 87i + 190 (the blue circle)

Alice will then a key() that will move her around the 37 integers until she arrives at 22i + 118 (green j-invariant) which will be the public key she sends to Bob

Bob will do the same, but instead will arrive at 334i +190 (yellow j-invariant) and that will be his public key he sends to Alice

When Alice combines her secret integer with Bob's key, she'll do a sequence of moves that will land her on the 234 (red j-invariant). This is the shared secret key $$g^{ab} = g^{ba}$$

And thus this is the simplified steps of what's going on in this post-quantum key exchange

In [2]:
using SymPy

In [17]:
i = symbols("i")
Pa = [100i + 248, 304i + 199]
Qa = [426i + 394, 51i + 79]

Pb = [358i + 275, 410i + 104]
Qb = [20i + 185, 281i + 239]

2-element Vector{Sym}:
  20⋅i + 185
 281⋅i + 239

In [18]:
#Alice generates a key picking a number {0... 15}
ka = 10
Sa = Pa.*ka + Qa; Sa

2-element Vector{Sym}:
 1426⋅i + 2874
 3091⋅i + 2069

In [34]:
#Bob generates a key picks a number {0... 26}
kb = 3
Sb = Pb.*kb + Qb

2-element Vector{Sym}:
 1094⋅i + 1010
  1511⋅i + 551

Alice's would then create a secret generator corresponding to her key:

$$S_A = \phi_B(P_A) + [k_A]\phi_B(Q_A)$$

While Bob's would be

$$S_B = \phi_A(P_B) + [k_B]\phi_B(Q_B)$$

<img src="pqdh.png" alt="pqdh" width="500"/>

Where

- densely dotted edges are representative of secret isogenies of one party
- loosely dashed edges are representative of  secret isogenies of the other party
- red curves are the common derived secret used for the common secret key
- yellow edges and curves are possible to compute but are not computed/used

### Security of SIDH
Classical*: claw finding $$ O(p^{1/4})$$
Classical: vOW $$ \frac{1}{\sqrt{m}}* p^{3/8}$$
where m is the number of processors run in parallel

Quantum*: $$O(p^{1/6})$$

In [20]:
#Let's first look at what the computation for this will get us
p = BigInt(2)^242
p

7067388259113537318333190002971674063309935587502475832486424805170479104

In [21]:
c1 = p^.25

1.630477228166597776543696475781563546110409536199107322341509459566896039850323e+18

if we're not sure a normal number of parallel processors, lets solve to see how many we might be abble to expect

In [25]:
using SymPy

In [26]:
m = symbols("m")

m

In [27]:
solveset(Eq(1/sqrt(m) * p^3/8, c1), m)

{7.323931180248121303426880622259254275817323967033174020674622978250045688496
e+398}

In [28]:
q1 = p^1/6

1.177898043185589553055531667161945677218322597917079305414404134195079850666672e+72

### B-Side SIDH

Supersingular Isogeny Diffie-Hellman using Twisted Torsion

Uses both the (p+1)-torsion and (p-1)-torsion

Making it so that Alice and Bob can work in the torsion corresponding to opposite sets of quadratic twists with no change to the protocol. 



### Security of B-Side SIDH

Classical*: Delfs-Galbraith $$ O(p^{1/2})$$
Quantum*: $$O(p^{1/4})$$

- For a level 1 security encryption $$2^{240} < p < 2^{256}$$ 

In [29]:
#lets see how the numbers will be when our p = 2^242 (same as prev)
p = BigInt(2)^242
print(p)

7067388259113537318333190002971674063309935587502475832486424805170479104

In [30]:
c2 = p^.5

2.658455991569831745807614120560689152e+36

In [31]:
q2 = p^.25

1.630477228166597776543696475781563546110409536199107322341509459566896039850323e+18

Let's compare numerically the values gotten from both, the numbers are massive so we will be finding the difference to better be able to understand the difference between them

In [32]:
#Classical
print(c2-c1)

2.658455991569831744177136892394091375456303524218436453889590463800892677658486e+36

In [33]:
#Quantum
print(q2-q1)

-1.177898043185589553055531667161945677218322597917079303783926906028482074122971e+72

Important to note that what worked for quantum did not work better for the classic computer and how that might lead to the way in which research is pursued within this field.

### Summary

We explored Lattice and Isogeny Cryptography today and saw the fascinating ways in which security is being strengthened. One of the most fascinating things with computer encryption is that just as people go to such lengths to protect information, others will find ways to decrypt and expose that information.

While it's not a good idea to decrypt illegally, it is fascinating to see how both push to new mathematical bounds.

These topics are still being researched and as such it's hard to know which category of cryptography will end up being the preferred method after more research has been done.

That means you can go forward and keep an eye out for new research, or go out there to research and find out for the rest of us! :)